In [ ]:
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import Dataset, DataLoader, sampler
import numpy as np
from copy import deepcopy
import matplotlib.pyplot as plt
import torch.optim as optim

#Cuda 혹은 cpu를 사용하시오.
############Write Your Code Here############
device = 'cuda'
############################################


#Custom_Dataset을 정의하시오.(10점)
class Custom_Dataset(Dataset):
    def __init__(self, X, y):
        #입력으로 들어온 X의 pixel값들을 0-1사이로 normalize하고 X의 shape을 (FB,C,H,W)로 변경하여 저장하여 self.X,self.y에 저장하시오.
        self.X = None
        self.y = None
        ############Write Your Code Here############
        self.X = (X / 255.0).reshape(-1, 3, 32, 32)
        self.y = y
        ############################################
        
    def __len__(self):
        #Custom_Dataset에 저장되어있는 총 data의 개수를 result에 저장하여 반환하시오.
        result = 0
        ############Write Your Code Here############
        result = len(self)
        ############################################
        return result
    
    def __getitem__(self, idx):
        #self.X, self.y 에서 idx에 맞는 data를 result_X,result_y에 저장하여 반환하시오.
        result_X,result_y = None,None
        ############Write Your Code Here############
        result_X = self.X[idx]
        result_y = self.y[idx]
        ############################################
        return result_X,result_y

    
#torch.nn을 사용하여 아래 함수들을 작성하시오. result는 nn.Layer중 하나이고 result를 반환함.(20점)
def batch_norm(dim,for_MLP=True):
    #for_MLP가 True일 시 MLP를 위한 BN Layer를 반환하고 False일 시 CNN을 위한 BN Layer를 반환함.
    ############Write Your Code Here############
    if for_MLP:
        result = nn.BatchNorm1d(dim)
    else:
        result = nn.BatchNorm2d(dim)
    ############################################
    return result

def fc_layer(in_dim,out_dim):
    #Fully Connected Layer(Dense Layer)
    ############Write Your Code Here############
    result = nn.Linear(in_dim, out_dim)
    ############################################
    return result

def conv_layer(in_ch,out_ch,kernel_size, stride=1, padding=0):
    #Convolutional Layer for image
    ############Write Your Code Here############
    result = nn.Conv2d(in_ch, out_ch, kernel_size, stride, padding)
    ############################################
    return result

def relu():
    #ReLU function
    ############Write Your Code Here############
    result = nn.ReLU()
    ############################################
    return result

def flatten():
    #Flatten the data
    ############Write Your Code Here############
    result = nn.Flatten()
    ############################################
    return result


#skip_connection(bn -> relu -> conv -> bn -> relu -> conv)를 따르는 Res_block을 만드시오.
#change_res가 True인 res_block을 통과한 feature map은 resolution이 2배 작아지고 channel의 깊이는 2배로 증가함. ex) 32*8*8 -> 64*4*4
#위의 경우에는 skip_connection의 dimension은 1*1 conv로 맞춰줌.
#change_res가 False인 Res_block을 통과한 feature map은 resolution과 channel의 깊이는 그대로 유지됨. ex) 32*4*4 -> 32*4*4(20점)
class Res_block(nn.Module):
    def __init__(self, input_channel, change_res):
        super(Res_block,self).__init__()
        self.change_res = change_res
        if change_res:
            ############Write Your Code Here############
            self.conv_block2 = nn.Sequential(
                batch_norm(input_channel),
                relu(),
                conv_layer(input_channel, 2*input_channel, 3, stride=2, padding=1)
            )
            self.skip_connection = conv_layer(input_channel, 2*input_channel, 1, stride=1, padding=0)
            ############################################
        else:
            ############Write Your Code Here############
            self.conv_block2 = nn.Sequential(
                batch_norm(input_channel),
                relu(),
                conv_layer(input_channel, input_channel, 3, stride=1, padding=1)
            )
            self.skip_connection = None
            ############################################
        ############Write Your Code Here############
        self.conv_block1 = nn.Sequential(
            batch_norm(input_channel),
            relu(),
            conv_layer(input_channel, input_channel, 3, stride=1, padding=1)
        )
        ############################################
    def forward(self,X):
        ############Write Your Code Here############
        Y = self.conv_block1(X)
        Y = self.conv_block2(Y)
        if change_res:
            X = self.skip_connection(X)
        X += Y
        ############################################
        return X

    
#Skip Connection을 이용하여 20개 이상의 layer를 가지고 테스트 셋에대하여 50%(70%) 이상의 성능을 주는 MLP를 만드시오.
#nn.ModuleList를 사용하면 많을 층의 layer를 쌓는데 용이함.(20점), conv layer 사용 x
class MLP(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MLP,self).__init__()
        ############Write Your Code Here############
        self.layer_1 = nn.Sequential(
            flatten(),
            fc_layer(input_dim, 100),
            relu(),
        )
        self.linears = nn.ModuleList([nn.Sequential(fc_layer(100, 100), relu()) for i in range(20)])
        self.layer_n = nn.Sequential(
            fc_layer(100, output_dim)
        )
        ############################################
    def forward(self,X):
        ############Write Your Code Here############
        X = self.layer_1(X)
        for layer in self.linears:
            X = layer(X)
        X = self.layer_n(X)
        ############################################
        return X
        
#Res_Block을 사용하여 테스트 셋에대한 75%(90%) 이상의 성능을 주는 CNN 모델을 만드시오.
#flatten전에 nn.AdaptiveAvgPool2d를 사용하면 dimension맞추기가 쉬움.(20점)
class CNN(nn.Module):
    def __init__(self, input_channel, class_number, block_number):
        super(CNN,self).__init__()
        ############Write Your Code Here############
        self.res1 = Res_block(3, False)
        self.res2 = Res_block(3, True)
        self.res3 = Res_block(6, False)        
        self.res4 = Res_block(6, True)
        self.fc = nn.Sequential(
            nn.AdaptiveAvgPool2d(16),
            flatten(),
            fc_layer(256, 512),
            fc_layer(512, class_number)
        )
        ############################################
    def forward(self,X):
        ############Write Your Code Here############
        X = self.res1(X)
        X = self.res2(X)
        X = self.res3(X)
        X = self.res4(X)
        X = self.fc(X)

        ############################################
        return X

#loader에 있는 모든 data들에 대한 정확도를 구하여 accuracy에 저장하여 accuracy를 return하는 함수를 구현하시오.(10점)
def evaluate(model, loader):
    model.eval()
    accuracy = 0
    total_example = 0
    correct_example = 0
    for data in loader:
        x,y = data
        x = torch.tensor(x, device = device)
        y = torch.tensor(y, device = device)
        ############Write Your Code Here############
        outputs = model(x.float())
        _, preds = torch.max(outputs, 1)
        correct_example += (preds == y).sum()
        total_example += preds.size(0)
        ############################################
    ############Write Your Code Here############
    accuracy = float(correct_example) / total_example
    ############################################
    model.train()
    return accuracy

#epoch마다 train_loader에 있는 batch들을 사용하여 모델을 학습하고
#epoch의 마지막 iteration에서는 모델의 validation accuracy를 확인하여 제일 높은 val. acc.를 가진 model을 best_model에 저장하고
#val_acc에는 매 epoch마다 구해진 validation accuracy를 저장하시오.
#running_loss에는 각각의 epoch에서 모든 batch의 loss를 다 더하여 저장하시오.
#모든 epoch의 validation accuracy를 val_acc에 저장하여 best_model과 val_acc를 return하는 함수를 구현하시오.(10점)
def train(model, epoches, train_loader, val_loader, optimizer, criteria):
    best_score = 0
    best_model = None
    batch_len = len(train_loader)
    val_acc = []
    for epoch in range(epoches):
        running_loss = 0
        for i,data in enumerate(train_loader):
            x,y = data
            x = torch.tensor(x, device = device)
            y = torch.tensor(y, device = device)
            ############Write Your Code Here############
            optimizer.zero_grad()

            outputs = model(x.float())
            _, preds = torch.max(outputs, 1)
            loss = criteria(outputs, y)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            ############################################
            
            #epoch의 마지막 iteration
            if i % batch_len == batch_len-1:
                print(f'{epoch+1}th iteration loss :',running_loss/batch_len)
                running_loss = 0
                ############Write Your Code Here############
                epoch_acc = 

                val_acc.append(0)
                if val_acc[epoch] > best_score:
                    best_score = val_acc[epoch]
                    best_model = copy.deepcopy(model.state_dict())
                ############################################
    return best_model, val_acc

In [ ]:
#(50점)
#Read the data
trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True)
testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True)

X_train, Y_train = trainset.data, np.array(trainset.targets)
X_test, Y_test = testset.data, np.array(testset.targets)


#앞서 정의한 Custom_Dataset과 DataLoader를 사용하여 train_loader,val_loader,test_loader를 정의하시오.
#Batch_size는 본인의 컴퓨터 사향에 맞게 변경하면 됨. Validation Set으로 Train Set의 20%를 사용함.
#Preprocessing
train_loader = None
val_loader = None
test_loader = None
batch_size = 64
############Write Your Code Here############
train_loader = DataLoader(Custom_Dataset(X_train, Y_train), batch_size=batch_size, 
                          sampler=sampler.SubsetRandomSampler(range(40000)), num_workers=4)
val_loader = DataLoader(Custom_Dataset(X_train, Y_train), batch_size=batch_size,
                          sampler=sampler.SubsetRandomSampler(range(40000, 50000)), num_workers=4)
test_loader = DataLoader(Custom_Dataset(X_test, Y_test), batch_size=batch_size, shuffle=False, num_workers=4)
############################################


#앞서 정의한 MLP,CNN을 사용하여 mlp_model,cnn_model을 정의하시오.
#Define the model
mlp_model = None
cnn_model = None
############Write Your Code Here############
mlp_model = MLP(3072, 10)
cnn_model = CNN(3, 10, 4)
############################################
mlp_model.to(device)
cnn_model.to(device)


#앞서 정의한 train함수를 사용하여 best_mlp, mpl_val_acc, best_cnn, cnn_val_acc를 구하시오.
#Train the model
best_mlp = None
mlp_val_acc = None
best_cnn = None
cnn_val_acc = None
############Write Your Code Here############
optimizer = optim.Adam(mlp_model.parameters(), lr=0.001)
criteria = nn.CrossEntropyLoss()
bet_mlp, mlp_val_acc = train(mlp_model, 5, train_loader, val_loader, optimizer, criteria)
optimizer = optim.Adam(cnn_model.parameters(), lr=0.001)
bet_cnn, cnn_val_acc = train(cnn_model, 5, train_loader, val_loader, optimizer, criteria)
############################################


#앞서 정의한 evaluate함수와 best_model들을 사용하여 mlp_acc, cnn_acc를 구하시오.
#Test Accuracy
mlp_acc = None  
cnn_acc = None 
############Write Your Code Here############
mlp_acc = evaluate(best_mlp, test_loader)
cnn_acc = evaluate(best_cnn, test_loader)
############################################
print('MLP accuracy:',mlp_acc)
print('CNN accuracy:',cnn_acc)


#앞서 구한 val_acc들을 사용하여 이해 가능한 그래프를 그리시오.
#Validation Accuracy Plot
############Write Your Code Here############
plt.plot(mlp_val_acc)
plt.plot(cnn_val_acc)
############################################

Files already downloaded and verified
Files already downloaded and verified


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:218: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:219: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


1th iteration loss : 2.3031842697143556
2th iteration loss : 2.3030340682983397
3th iteration loss : 2.3028968070983886
4th iteration loss : 2.302853929901123
5th iteration loss : 2.3027852333068846


ValueError: ignored